# 1. Import and Install Dependencies

# Run installs

In [4]:
!pip install mediapipe
!pip install tensorflow==2.14.0
!pip install opencv-python
!pip install mediapipe
!pip install matplotlib

# Run preps

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow import keras
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

def rename_dirs(starting_pos):
    all_data_dir_path = os.path.join(os.getcwd(), 'MP_Data')
    
    activities_dirs = [name for name in os.listdir(all_data_dir_path) if os.path.isdir(os.path.join(all_data_dir_path, name))]
    
    if not len(activities_dirs):
        return
    
    # Get the current directory
    for activity_dir_name in activities_dirs:
        banned_names = ["move_left", "move_right", "pc_lock", "pc_on"]
        if activity_dir_name in banned_names:
            continue
        activity_dir_path = os.path.join(all_data_dir_path, activity_dir_name)
    
        # List all directories in the current location
        activity_dir_enumerated_dirs = [num_name for num_name in os.listdir(activity_dir_path) if os.path.isdir(os.path.join(activity_dir_path, num_name))]

    
        if starting_pos > 0:
            activity_dir_enumerated_dirs.sort(reverse=True, key=int)
        else:
            activity_dir_enumerated_dirs.sort(reverse=False, key=int)

        # Rename the directories
        for directory_num_name in activity_dir_enumerated_dirs:
            new_name = int(directory_num_name) + int(starting_pos)
            os.rename(os.path.join(activity_dir_path, str(directory_num_name)), os.path.join(activity_dir_path, str(new_name)))

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

colors = [(245,117,16), (117,245,16), (16,117,245), (1,100,200), (10,50,70), (1,150,30)]

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
# actions = np.array(['pc_on', 'pc_lock', 'move_left', 'move_right', 'show_destkop', 'mute'])
actions = np.array(['pc_on', 'pc_lock', 'move_left', 'move_right', 'show_destkop', 'mute'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Run test keypoints

In [2]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# Manage folders

In [3]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [45]:
rename_dirs(0)

# Collect Keypoint Data for Training

In [5]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(700)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [8]:
cap.release()
cv2.destroyAllWindows()

# Build and Train LSTM Neural Network

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []

# Define the number of sequences for each action
no_sequences_per_action = {
    'pc_on': 30,
    'pc_lock': 30,
    'move_left': 30,
    'move_right': 30,
    'show_desktop': 30,
    'mute': 30
}

for action in actions:
    no_sequences = no_sequences_per_action.get(action, 30)  # Default to 30 sequences if not specified
    
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

y = to_categorical(labels).astype(int)
X = np.array(sequences)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
res = [.7, 0.2, 0.1]
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [11]:
np.array(sequences).shape

(1830, 30, 258)

In [47]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
55/55 [==============================] - 9s 99ms/step - loss: 0.9233 - categorical_accuracy: 0.6646
Epoch 2/2000
55/55 [==============================] - 5s 97ms/step - loss: 0.3243 - categorical_accuracy: 0.9074
Epoch 3/2000
55/55 [==============================] - 5s 99ms/step - loss: 0.2043 - categorical_accuracy: 0.9384
Epoch 4/2000
55/55 [==============================] - 5s 98ms/step - loss: 0.1187 - categorical_accuracy: 0.9695
Epoch 5/2000
55/55 [==============================] - 5s 99ms/step - loss: 0.0834 - categorical_accuracy: 0.9781
Epoch 6/2000
55/55 [==============================] - 6s 100ms/step - loss: 0.0814 - categorical_accuracy: 0.9770
Epoch 7/2000
55/55 [==============================] - 5s 98ms/step - loss: 0.1058 - categorical_accuracy: 0.9741
Epoch 8/2000
55/55 [==============================] - 5s 98ms/step - loss: 0.0797 - categorical_accuracy: 0.9816
Epoch 9/2000
55/55 [==============================] - 5s 99ms/step - loss: 0.0560 - categorical

# Save/Load model 

In [48]:
model.save('my_model.keras')

model.load('my_model.keras')

In [6]:
model = keras.models.load_model('150rc4fun.keras')

# Test in Real Time

In [8]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.95

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(res)
            #print(actions[np.argmax(res)])
            
            # Calculate recognition percentage
            recognition_percentage = round(100 * res[np.argmax(res)], 2)
            print(f"Gesture recognition percentage: {recognition_percentage}%")
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #print(res)
            #print(colors)
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 0s 24ms/step
[0.8238499  0.00917807 0.15517357 0.01179838]
Gesture recognition percentage: 82.38%
1/1 [==============================] - 0s 23ms/step
[0.80037093 0.01187133 0.17465901 0.01309861]
Gesture recognition percentage: 80.04%
1/1 [==============================] - 0s 23ms/step
[0.7944673  0.01241267 0.18012258 0.01299753]
Gesture recognition percentage: 79.45%
1/1 [==============================] - 0s 24ms/step
[0.80726725 0.01195214 0.17005692 0.01072371]
Gesture recognition percentage: 80.73%
1/1 [==============================] - 0s 24ms/step
[0.8326016  0.01055455 0.14884986 0.00799387]
Gesture recognition percentage: 83.26%
1/1 [==============================] - 0s 25ms/step
[0.8630661  0.00863844 0.12219212 0.00610332]
Gesture recognition percentage: 86.31%
1/1 [==============================] - 0s 25ms/step
[0.89492506 0.0063515  0.09383632 0.00488708]
Gesture recognition percentage: 89.49%
1/1 [==============================] - 0s